In [1]:
import geojson
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import numpy as np 
from pandas import DataFrame
import geopandas as gpd
from shapely.geometry import Point, Polygon
%matplotlib inline
pd.set_option('display.max_colwidth', -1)
from IPython.display import clear_output
clear_output()


In [2]:
def venue2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["longitude"],
                                                    X["latitude"])),
                            properties=dict(venueId=X["venueId"],
                                            venueCategory=X["venueCategory"],
                                            venueCategoryId=X["venueCategoryId"],
                                            timezoneOffset=X["timezoneOffset"],
                                            utcTimestamp=X["utcTimestamp"],
                                            userId=X["userId"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

        
def screen2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["longitude"],
                                                    X["latitude"])),
                            properties=dict(id=X["id"],
                                            venue=X["venue"],
                                            City=X["City"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [4]:
user_visit_data_point = pd.read_csv('/folder_name/input_data.csv', engine='python')
user_visit_data_point = user_visit_data_point.head(200)


In [5]:
user_visit_data_point_geo = geopandas.GeoDataFrame(user_visit_data_point, geometry=gpd.points_from_xy(user_visit_data_point.longitude,
                                                                                user_visit_data_point.latitude))

In [43]:
user_visit_data_point_geo.crs = {'init' :'epsg:4326'}

In [7]:
#user_visit_data_point_geo.drop(['latitude', 'longitude'], axis=1, inplace=True)
clear_output()

In [28]:
places_data = pd.read_csv('/folder_name/input_data_.csv', engine='python')
clear_output()

In [44]:
nyc_cities = ["New York"]

In [45]:
places_data_nyc= places_data[places_data['City'].isin(nyc_cities)]

In [46]:
places_data_nyc_geo = geopandas.GeoDataFrame(places_data_nyc, geometry=gpd.points_from_xy(places_data_nyc.longitude,
                                                                                places_data_nyc.latitude))

In [47]:
places_data_nyc_geo.drop(['latitude', 'longitude'], axis=1, inplace=True)
clear_output()

In [33]:
filename = "input_file.geojson"
screen2geojson(places_data_nyc, filename)
places_data_nyc_geo = gpd.read_file(filename)

In [34]:
def create_polygon_by_geo(gdf):
    gdf.crs = {'init': 'epsg:4326'}
    gdf = gdf.to_crs({'init': 'epsg:3857'})
    gdf['geometry'] = gdf['geometry'].apply(lambda x: x.buffer(50).envelope)
    gdf = gdf.to_crs({'init': 'epsg:4326'})
    return gdf

In [35]:
places_data_present_geo = create_polygon_by_geo(places_data_nyc_geo)

In [36]:
places_data_present_geo.to_crs({'init': 'epsg:4326'})
clear_output()    

In [37]:
venue_data_geo_venue = gpd.sjoin(places_data_present_geo, user_visit_data_point_geo,  how="inner", op="within")

In [48]:
filename = "/folder_name/output_data.geojson"
places_data_present_geo.to_file(filename, driver="GeoJSON")
clear_output()

In [49]:
filename = "/folder_name/output_data.geojson"
user_visit_data_point_geo.to_file(filename, driver="GeoJSON")

In [41]:
from keplergl import KeplerGl 
map_2 = KeplerGl(height=500)

map_2.add_data(data=places_data_present_geo, name="screens")

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [25]:
map_2.add_data(data=user_visit_data_point_geo, name="venues")
map_2

KeplerGl(data={'screens': {'index': [321, 396, 522, 765, 806, 1256, 1470, 2046, 2352, 2501, 2637, 2992, 3171, …